<a href="https://colab.research.google.com/github/ajaykrishna2013/NLP/blob/main/w3_Quiz_NLP_NLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=brown><i><b>Copyright Information:</b></i> All course material is copyrighted by the author and Stanford University with all rights reserved. The material may not be reproduced or distributed without an explicit written permission of the author. Stanford University also has a policy that one should know before audio/video recording. Do not post material online.</font>

# **w3 Quiz. NLP and NLU**

Make a copy of this Colab Notebook with the **starter code** below and continue building your solution **in Colab** (not another Python environment) to assure an exact environment and matching solutions.

Prof. Melnikov's video in module 3 demonstrates several ways to **tokenize sentences** and **words**. Here we evaluate their effectiveness in reducing the dimensionality of the vocabulary, while maintaining the "quality" of the tokens. We define "quality" with a binary function with *high* quality for words found in some commonly accepted vocabulary (such as [**Brown corpus**](https://www.nltk.org/book/ch02.html) from **NLTK library**). Other words are considered as *low* quality. This will require Python **set operations**, since we need to check whether a given word is in the vocabulary set or not.

Most evaluations here are done one the first 100 posts from each of the [**20 Newsgroups corpus**](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) in **Scikit-Learn library**. We then use the average metric (**median**, in this case) to compute the final quantity, which you will submit to Canvas Quiz.

This quiz is a sequence of small projects, requiring computation of values. Do not round any values. If stuck for 10-15 minutes, raise a question in the w3 Discussion Q&A. See Syllabus for **Posting Guidelines**. In particular, describe your situation and what you have tried **without posting your code**.

To ease code readability  and avoid confusion, we use the following prefix convention for key variables: 
1. `s`=string, `n`=number, `b`=Boolean
1. `Ls`=list of strings, `Ln`=list of numbers, `Ss`=set of strings, `As`=NumPy array of strings, `Ds`=dictionary of string values
1. `LLs`=list of lists of strings, `LTs`=list of tuples of strings, etc.
1. `df`=Pandas dataframe or series, 

<p><font color=gray><i>Hint</i>: Refer to videos in Module 3 and some Python refresher videos of Corey Schafer.</font>

In [ ]:
!pip -q install contractions   # quietly install contractions package
# allows multiple outputs from a single Colab code cell:
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

import sys, matplotlib.pylab as plt, re, platform, matplotlib
import numpy as np, pandas as pd, nltk, sklearn, spacy, unicodedata, contractions 
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
tmp = nltk.download(['brown', 'stopwords','punkt','wordnet'], quiet=True) # See https://www.nltk.org/book/ch02.html
LsStopWords = nltk.corpus.stopwords.words('english')
from nltk.corpus import brown

# Increase viewable area of Pandas tables, NumPy arrays, plots
pd.set_option('max_rows', 5, 'max_columns', 500, 'max_colwidth', 1, 'precision', 2)
np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
plt.rcParams['figure.figsize'] = [16, 4]

def LoadNews(cat=['sci.space'], TopN=100):
    '''Function to load a string of news posts for the specified categories. Returns: TopN concatenated news'''
    Rem = ('headers', 'footers', 'quotes')  # remove these fields from result set
    bunch = fetch_20newsgroups(categories=cat, subset='test', shuffle=False, remove=Rem)
    return '\n'.join(bunch.data[:TopN])  # save first 100 posts concatenated as a single string.

# See doc: https://scikit-learn.org/stable/datasets/index.html#newsgroups-dataset
# We preload string variables containing concatenated news posts 
sNews = LoadNews(['sci.space'])   # a string. news from space ;)  
LsTgtNames = list(fetch_20newsgroups().target_names)   # names of 20 newsgroups

pso = nltk.stem.PorterStemmer()       # instantiates Porter Stemmer object
wlo = nltk.stem.WordNetLemmatizer()   # instantiates WordNet lemmatizer object
SsBrownVcb = set(brown.words())       # Vocabulary of 56057 words in Brown Corpus

# store sentence tokenizers' results as a list of lists or strings:
nlp = spacy.load('en_core_web_sm')
LLsST =  [sNews.split('. ')] \
    + [nltk.sent_tokenize(sNews)] \
    + [nltk.tokenize.PunktSentenceTokenizer().tokenize(sNews)] \
    + [[n.text for n in nlp(sNews).sents]]

# store word tokenizers' results as a list of lists of strings
LLsWT = [sNews.split()] \
    + [nltk.RegexpTokenizer(pattern=r"\s+", gaps=True ).tokenize(sNews)] \
    + [nltk.RegexpTokenizer(pattern=r"\s+", gaps=True ).tokenize(sNews)] \
    + [nltk.WhitespaceTokenizer().tokenize(sNews)] \
    + [nltk.RegexpTokenizer(pattern=r"\w+", gaps=False).tokenize(sNews)] \
    + [nltk.word_tokenize(sNews)] \
    + [nltk.TreebankWordTokenizer().tokenize(sNews)] \
    + [[t.text for t in nlp(sNews)]] \
    + [nltk.tokenize.toktok.ToktokTokenizer().tokenize(sNews)] \
    + [nltk.WordPunctTokenizer().tokenize(sNews)]

     |████████████████████████████████| 266kB 6.8MB/s 
     |████████████████████████████████| 327kB 11.7MB/s 


In [ ]:
# Example of news article
pprint(sNews[0:100])  

("I'm afraid I was not able to find the GIFs... is the list \n"
 'updated weekly, perhaps, or am I just mis')


In [ ]:
# Examples of parsed sentences by different tokenizers
[{i:LLsST[j][i][:100] for i in [0,1]} for j in range(len(LLsST))]

[{0: "I'm afraid I was not able to find the GIFs..",
  1: 'is the list \nupdated weekly, perhaps, or am I just missing something?\n\nThe forces and accelerations '},
 {0: "I'm afraid I was not able to find the GIFs... is the list \nupdated weekly, perhaps, or am I just mis",
  1: 'The forces and accelerations involved in doing a little bit of orbital\nmaneuvering with HST aboard a'},
 {0: "I'm afraid I was not able to find the GIFs... is the list \nupdated weekly, perhaps, or am I just mis",
  1: 'The forces and accelerations involved in doing a little bit of orbital\nmaneuvering with HST aboard a'},
 {0: I'm afraid I was not able to find the GIFs... is the list 
  updated weekly, perhaps, or am I just missing something?
  , 1: The forces and accelerations involved in doing a little bit of orbital
  maneuvering with HST aboard are much smaller than those involved in
  reentry, landing, and re-launch.  }]

In [ ]:
# Examples of parsed words by different tokenizers
[LLsWT[i][0:9] for i in range(len(LLsWT))]

[["I'm", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find', 'the'],
 ["I'm", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find', 'the'],
 ["I'm", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find', 'the'],
 ["I'm", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find', 'the'],
 ['I', 'm', 'afraid', 'I', 'was', 'not', 'able', 'to', 'find'],
 ['I', "'m", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find'],
 ['I', "'m", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find'],
 ['I', "'m", 'afraid', 'I', 'was', 'not', 'able', 'to', 'find'],
 ['I', "'", 'm', 'afraid', 'I', 'was', 'not', 'able', 'to'],
 ['I', "'", 'm', 'afraid', 'I', 'was', 'not', 'able', 'to']]

## **P1. Sentence Tokenizers**

Compute the mean (average) length of a sentence (as a count of characters) for each tokenizer and save it to `LnMeanSentLen`. Compute the difference between maximal mean length and minimal mean length.

<p><font color="gray"><i>Takeaway:</i> 
Notice the drastic difference in sentence lengths among these tokenizers. Wow! Investigate the shortest and longest sentences. Are they parsed correctly? What sentence separator can be used to tokenize these correctly? Which sentence tokenizer appears most/least reliable. Which one is fastest (if you tried timing these)? Are there tuning parameters for poorly performing parsers to improve their tokenization results? How would you pre-process the corpus to improve sentence tokenization?
<br><i>Hints</i>: The answer is in [100, 200] interval. It might be easier if you use list comprehensions and <code>np.mean()</code> function</font>

In [ ]:
# Example. LnMeanSentLen=[6.0, 14.0]. Then max - min of these values yields 8. 
LLsST_ = [['5char','7  char'],['5char','7  char','a sentence with 30 characters ']]

## **P2. Word Tokenizers**

Consider а list of lists of word strings, <code>LLsWT</code>, created above. <b>Compute</b> <code>abs(nMaxWordLength-nMaxWordCount)</code>

1. `nMaxWordCount` = max count of tokens among each tokenizer.
1. `nMaxWordLength` = max character length among all words from all tokenizers.

--------------

<font color=gray><i>Takeaway:</i> Are the longest and shortest word tokens meaningful? If not, should we avoid these or pre-process these in some way?
<br><i>Hint:</i> The answer is in the [23000, 24000] interval. It is easier to process <code>LLsWT</code> via loops, list comprehensions or Pandas dataframes.
</font>

In [ ]:
# Example: nMaxWordCount=9 and nMaxWordLength=6. The absolute difference is |9-6| or 3
LLsWT_ =  [['I', "'m", 'afraid'], ['I', "'", 'm', 'afraid', 'I', 'was', 'not', 'able', 'to']]

## **P3. Word Tokenizers - Largest Word**

Consider a list of lists <code>LLsWT</code> created above. <b>Compute</b> the length of the longest token containing <a href=http://sticksandstones.kstrom.com/appen.html>ASCII</a> letters, i.e. any of a-z or A-Z.

<p><i>Takeaway:</i> What additional preprocessing would you include to avoid semantic-less word tokens in your resulting vocabulary?

-----------

<font color=gray><p><i>Hint:</i> The answer is in the [0, 200] interval. You may need to flatten a list of lists of strings into just a list of strings for convenience. Then remove words that do not contain any ASCII letters. This can be done with the `search()` method from `re` object and `[a-zA-Z]` pattern.

In [ ]:
# Example: The longest word token containing `[a-zA-Z]` is `'here.'` and has length 5.
LLsWT_ = [['I', "'", 'm', 'here', '. (2021)'], ['I\'m', 'here.', '(2021)']]

## **P4. Dimension Reduction: Lower Casing**

Here we compare the effectiveness of lower casing of a text corpus. Consider news text from `LoadNews(['rec.motorcycles'])` with original and lower casing. Compute the **percentage decrease in vocabulary size** of word tokens with and without lowercase pre-processing. Use `nltk.WordPunctTokenizer()` object to parse into word tokens. Then do the same for *each newsgroup element* of <code>LsTgtNames</code> list. Submit the median of these quantities.

If original and processed vocab sizes are <code>a</code> and <code>b</code>, then we want <code>(a-b)/a*100</code> as the percent decrease in vocab size.

**Toy example 1:** A string <code>"r you R user?"</code> has a vocabulary (i.e. unique word tokens) of 4 words; and its lower-case counterpart <code>"r you r user?"</code> has a vocabulary of 3 words (because "R" was replaced with "r"). So, the percentage decrease in vocabulary is (4-3)/4*100=25%. Submit 25

**Example 2:** You should observe these intermediate results. Here the rows ordered by `%improvement`:

|group|#orig_words|#lowcase_words|%improvement|
|-----|-----------|--------------|--------|
|rec.motorcycles|2712|2484|8.41|
|comp.sys.mac.hardware|2384|2183|8.43|

<p><font color="gray"><i>Takeaway:</i> Notice a healthy reduction in vocabulary size across these newsgroups by lower casing the text. Also, a median is a better measure of centrality (or average) than mean because the former is robust to extreme values (or outliers). FYI: <code>nltk.WordPunctTokenizer()</code> is fast, but offers poorer quality than Spacy's <code>nlp</code> model.

-------

<font color=gray><i>Hint:</i> First write your code for a single news group; then wrap it into a function; then call this function for every newsgroup. It's simpler to apply lower casing before tokenization. Note vocabulary is always a container of unique words.

<font color=gray>Just like a real vocabulary or dictionary, a digital vocabulary also contains unique words only. A big challenge in NLP is handling (storing and processing) huge vocabularies. In classical NLP we normalize words to reduce the vocabulary size with a minimal "loss of information". So, if your vocabulary contains 1 million words, then each word is essentially a 1 million dimensional one-hot vector (of zeros and a single 1 identifying the unique position of the word). If we reduce the vocabulary to 100K words, then each word is a 100K dimensional one-hot vector. Naturally, with shorter vector representations we can fit more words in compute memory and do more NLP magic. Pre-processing helps us reduce the vocabulary size.


## **P5. Dimension Reduction: Contraction Expansion (CE)**

Similar to P4, measure the percent decrease in vocabulary size across multiple news groups due to the application of CE with the <code>contractions</code> package (without modifying). Apply CE before tokenization. See w3 Colab notebook.

**Example:** You should observe these intermediate results. Here the rows ordered by `%improvement`:


|group|#orig_words|#CE_words|%improvement|
|--|--|--|--|
|talk.politics.mideast|5704|5685|0.33|
|soc.religion.christian|5101|	5083|	0.35|

<p><font color="gray"><i>Takeaway:</i> As expected, the CE is not as effective, but is complementary to lower casing and other techniques. Typically,we create pre-processing pipelines, where the order matters. For example, given "I'm here", CE+parsing yields <code>['I','am','here']</code>, while parsing+CE yields <code>['I am', 'here']</code>.

---------------------

<font color="gray">Notice that we are measuring the percent decrease in vocabulary size due to application of contraction expansion only. The previous preprocessing of lower casing is not used here, since it would make it more difficult to measure the effect of expansion along.</font>


## **P6. Dimension Reduction: Stopwords Removal**

Similar to P4, measure the percent decrease in vocabulary size across multiple news groups due to the removal of stop words. Use <code>LsStopWords</code> defined above. Remove stopwords after tokenization. See w3 Colab.

**Example:** You should observe these intermediate results. Here the rows ordered by `%improvement`:


|group|	#orig_words|	#important_words|	%improvement|
|--|--|--|--|
|talk.politics.mideast|	5704|	5568|	2.38|
|comp.graphics|	4971|	4842|	2.60|

<font color=gray><i>Takeaway:</i> Are you surprised by the average percent of stop words in these corpora?

## **P7. Dimension Reduction: Normalizing Accented Characters (NAC)**

Similar to P4, measure the percent decrease in vocabulary size across multiple news groups due to the removal of accent marks as we did in Module 3. Use <code>LsStopWords</code> defined above to retrieve smaller vocabularies. First do NAC, then tokenize. See w3 Colab. 

**Example:** You should observe these intermediate results. Here the rows ordered by `%improvement`:


|group|	#orig_words|	#NAC_tokens|	%improvement|
|--|--|--|--|
|alt.atheism|	4928|	4928|	0.0|

<font color=gray><i>Takeaway:</i> Does the answer surprise you? Why do you think you are seeing this percentage reduction?

## **P8. Dimension Reduction: Porter Stemmer vs WordNet Lemmatizer**

Similar to P4, we compare the effectiveness of stemmers and lemmatizers in reducing the vocabulary size. 

Apply the stemmer <code>pso</code> (defined above) to the tokenized words of each news group (as we did in Module 3 video and Jupyter Notebook). Then compute the median percent decrease in vocabulary size. Then apply the lemmatizer <code>wlo</code> (defined above) the same way. Then submit the largest of the two percentage decrease values.

**Example:** You should observe these intermediate results. Here the rows ordered by `%improvement`:

|group|#orig_words|#pso_words|%improvement|
|-|-|-|-|
|rec.sport.hockey|4026|3271|18.75|
|rec.motorcycles|2712|2158|20.43|

|group|#orig_words|#wso_words|%improvement|
|-|-|-|-|
|misc.forsale|3827|3708|3.11|
|rec.sport.hockey|4026|3897|3.20|

<font color=gray><i>Takeaway:</i> Notice the more aggressive normalizer. Do you think it produces higher quality words (that can still be found in a common English dictionary)?

## **P9. Dimension Reduction: Measure Quality of Stems/Lemmas**

Here we are quantitatively evaluating the quality of contributions to vocabulary from **stemming** and **lemmatization**. Many of the stemmed and lemmatized words may not be proper English words. We assess whether stems and lemmas are spelled correctly by checking whether they appear in the set of 56K word vocabulary from **Brown corpus**, which is <code>SsBrownVcb</code> set object created above.

<p>Given one newsgroup corpus, say <code>'misc.forsale'</code>, compute <code>vocab_orig</code> set of tokens from the original text and <code>vocab_pso</code> set of stemmed tokens using <code>pso</code> object. Then compute a set of newly formed word tokens, <code>new_tokens_pso</code>, that were not in the original set. Now, how many of these new stems are in the Brown vocabulary? Compute the percent of new stems which are also found in Brown corpus (vs all new stems from <code>pso</code>). Let's call this quality metric <code>accuracy_pso</code>.

<p>Now, compute <code>accuracy_wlo</code> similarly, but with a lemmatizer object <code>wlo</code> created above. 

<p>Compute the absolute difference, <code>abs(accuracy_wlo-accuracy_pso)</code> and call it <code>abs_acc_diff</code>.

<p>Finally, compute the median of all <code>abs_acc_diff</code> metrics across all newsgroups in <code>LsTgtNames</code> list.

<p><font color="gray"><i>Takeaway:</i> Notice the drastic difference in quality between two techniques.

---------------

<i>Hint:</i> You will need to use set operations here. Sets are extremely effective for testing memberships of members of one group in another group. See Corey Schafer video on sets, if you need a refresher. </font>

<hr>

**Toy example 1:** Say you have a sentence "NLP is awesomely exciting", which tokenizes to <font color=blue>["NLP", "is", "awesomely", "exciting"]</font>, which stem to (for example) <font color=blue>["NLP", "is", "awesome", "excit"]</font>. Now, there are 2 new words and, suppose, only one of them is correct. That is <font color=blue>"awesome"</font> is a new word that is correct and <font color=blue>"excit"</font> is incorrect. 

<p>So, we only have 50% of the new words, which are correct. Then we would expect lemmatization to yield 100% of new words to be correct. How do we determine whether the word is correct or not? We could spell-check, but it's slow. We could also check if it's in some dictionary of words that we consider correct, such as Oxford Dictionary, Wikipedia, etc. Well, often we just use Brown vocabulary as that ground-truth set of words. So, in this exercise we need to compute the percent of new words found in Brown corpus (vs all new stems from pso).

<p>Further clarification:

1. Let S:= set of new words resulting from stemming (i.e. new stems)
1. Let B:= set of all words in Brown vocabulary (these are unique words)
1. Let BS:= the intersection of the B and S. That is all new words found in Brown corpus
1. len(BS) is the size of this intersection, i.e. the count of elements in the BS set.
1. We need to find len(BS)/len(S) as a percentage.

<p>Finally, we compute the absolute difference between quality metrics, scale it up to all newsgroups, and then pick the median for submission.

You should observe these intermediate results. Here the rows ordered by `abs_acc_diff`:

|newsgroups|acc_diff_pso|acc_diff_wlo|abs_acc_diff|
|--|--|--|--|
|misc.forsale|30.07|80.49|50.42|
|rec.motorcycles|26.52|79.00|52.48|